# Introduction
**Fair algorithms are designed to remove unwanted impacts of protected attributes such as gender or ethnicity  on automatic decisions. 
Early fairness aware machine learning methods viewed fairness from a static point of view and did not consider the delayed impact of decisions on the future.
Recently, long term impacts and dynamics of decisions moved into the focus and the long term effects of decisions have been studied. 
Some of the key insight are, that decisions that appear to be fair from a static point of view may have negative long term impact.   
This student project develops a framework to simulate the influence of static decision rules on long term fairness under different assumptions regarding the dynamics of the data generation. The framework is designed to work with the [AIF360](https://aif360.mybluemix.net/) toolkit, in order to provide out of the box access to fairness metrics and algorithms.
Data is generated sequentially assuming an impact of previous features and predictions on the current time. Two different 
assumptions about the data generation are implemented:**

**1. Only individuals benefit from positive decisions.**

**2. The whole group sharing the protected attribute benefits from positive decisions.**

**Depending on the assumptions regarding the data generation static decision rules have different impact on the long term dynamics.**

# 1. Fairness in Machine Learning
Fairness in static settings is usually evaluated on several data sets  that are public available. Prominent examples are for instance the german credit data set [GerCred](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29) or the compas data set [Compas](https://github.com/propublica/compas-analysis/). 

For dynamic fairness there is no such data available and it is not trivial to collect it (new data has to change depending on the decisions made). Most works in dynamic or long term fairness therefore make assumptions about the dynamics of the data. These assumptions can be seen as grounded in sociology and social learning. The behavior of an individual is not solely determined by internal factors such as cognitive skills, but also by the environment.

The two key mechanisms assumed to determine dynamics in this project are that individuals learn from experiences and that the social environment serves as role model. For the first factor, learning from experiences, consider a company hiring an employee. After being hired, this employee will increase the skill for this job by experience. If someone was not skilled initially, the right treatment might make this person qualified. An example for social learning for instance is, that children from parents who graduated from university are more likely to graduate themselves [DZHW](https://www.dzhw.eu/publikationen/index_html). In this case, the parents are assumed to serve as role model.

These assumptions are implemented in form of a sequential data generator, sampling new data points given data from previous time steps. The provided framework is meant to be used for estimating the effects of static decision rules on dynamic fairness under two different assumptions regarding the data generation. The assumptions are that either only individuals with  positive predictions benefit from a decision or that the whole group sharing the protected attribute benefits. Details about the data generation are explained in next notebook. To estimate the effects of a decision rule, a baseline data pipeline provides data generated under the assumption that all individuals in the previous time steps were assigned a positive predictions.

Note, that the baseline data serves as metric here and therefore long term fairness is implicitly considered to be measured by the overall number of positive labeled individuals in future. This by it self is not really suitable as fairness measure, but this is discussed at the end.

Problems and pitfalls regarding the data assumptions and the baseline data generation are as well discussed at later, and an improved data generation algorithm is proposed.

The jupyter notebooks of chapter two (starting with 2_) describe the data generator and the framework.
The next part recaps some definitions of machine learning before introducing the notation for long term fairness.

## 1.0 Notation
Fair machine learning in this setup is considered as a sequential process at discrete time steps $t$. A superscript $^{(t)}$ will indicate when features at fixed time steps $t$ are referred to. Furthermore, bold capital letters will be used to distinguish the aggregated data and the data at one time step $t$. The following part introduces the general notation for machine learning at a fixed time steps $t$ (neglecting the superscript $^{(t)}$).

Features $$X \in \mathbb{R}^{n \times m}$$

True labels $$y \in \mathbb{R}^{n} $$

Predictions $$\hat{y} \in \mathbb{R}^{n} $$

Decision function $$d~: X \rightarrow \hat{y}$$

Error function $$\mathcal{L}~: \hat{y} \times y \rightarrow \mathbb{R}$$

The optimal decision function $d^*$ is found by minimizing the error function:

$$\min_d \mathcal{L}(\hat{y}, y)$$


## 1.1 Static Fairness Setup
In fairness aware machine learning, some features $A \in \mathbb{R}^{n \times k} $ are considered protected (e.g. describing ethnicity, gender or age). Here, the protected features are not contained in the features $X$ ($A \notin X$). The new decision function $d'$ now is a function of two parameters:

$$d' : X \times A \rightarrow \hat{y}$$

$d'$ must satisfy fairness for the protected features $A$ with respect to some fairness measure $f$. Two example measures are discussed in 1.2.

The next part briefly scatches, how such a decision function $d'$ can be found. However, because no functions are explicitly trained it is not  discussed in detail.


### 1.1.1 Finding Fair Decision Rules

Three different approaches can be used for constructing fair decision rules:

#### 1. Pre-Processing:
The features $X$ and $A$ are changed such that the resulting data $X_f, A_f$ has no unfairness. 

#### 2. In-Processing:
The learning is adopted, such that the learned decision function $d'$ is fair. This can for instance be achieved by extending the loss function with a penalty term w.r.t. to the desired definition of fairness.

#### 3. Post-Processing:
A learned decision function $d$ is adjusted such that the output of the new function $d': ~g \circ d$ is fair.

## 1.2 Measuring Static Fairness
Two metrics are used as examples in this project and discussed here. There are several ways to describe fairness in terms of a function $f$. In general, the metric $f$ can be any function mapping a subset $ \mathcal{M} \in \{X^{(t)},A^{(t)},Y^{(t)},\hat{Y}^{(t)} \}$ to a real value representing the fairness:

$$ f: \mathcal{M} \in \{X,A,Y,\hat{Y}\} \rightarrow \mathbb{R}$$

Metrics can roughly be categorized using the input signatures. Each metric uses the protected features $A$ and predictions $\hat{Y}$.

If metrics take error rates into consideration, they will additionally need the true labels $Y$. If $X$ is required as input, the metric would care about similarity of individuals.

Again, metrics are not covered in detail here [fairmlBook](https://fairmlbook.org/pdf/classification.pdf). 

### 1.3.1 Disparate Impact (or Demographic Parity)
Disparate Impact is one of the first metrics considered in fair machine learning. It only considers the acceptance rates and requires that the number of positive labeled individuals between all protected groups must roughly be the same.

Using probability distributions it can be written as:

$$P(\hat{Y}^{(t)} = 1 | A = a_i) = P(\hat{Y}^{(t)} = 1 | A = a_j) ~~ \forall i, j $$ 

A notation that can be implemented as function  $f: \hat{Y}^{(t)} \times A^{(t)} \rightarrow \mathbb{R}$ assuming a binary attribute $A \in \{0, 1\}$ would be:

$$f(\hat{Y}^{(t)}, A^{(t)})= \sum_{i =0}^{n} \frac{ \mathbf{1}_1(a_i^{(t)}) \cdot y_i^{(t)}}{ \mathbf{1}_0(a_i^{(t)}) \cdot y_i^{(t)}} $$

Here, $1$ is considered to be the positive label, $\mathbf{1}_1(a_i)$ is the indicator function for the positive class and $\mathbf{1}_0(a_i)$ for the negative class respectively. 

In a real world example, disparate impact states that if a company wants to hire 10 people from two groups it must hire 5 people from each group.
This is, however, very restrictive, therefore a relaxation is the so called $p$-rule. It states that acceptance rates do not need to be equal but only have to satisfy some fraction $p$.

Limits of disparate impact are for instance laziness. A decision function trying to satisfy disparate impact might  accept individuals with a true negative label simply to satisfy disparate impact. Another limitation often mentioned is, that disparate impact might rule out the perfect classifier.

### 1.3.2  Error based Metrics (Equal Opportunity or Separation)
Error based metrics consider the true label $Y$.
In general, they require the error rates to be minimized. Depending on the scenario one might by interested in overall error rates, false positive/negatives, recall or omissions. 

The definition as probability distribution would be:
$$P(\hat{Y}^{(t)} = \hat{y} | Y = y, A = a_i ) = P(\hat{Y}^{(t)} = \hat{y} | Y=y, A = a_j) ~~ \forall i, j $$ 

One limit if error based metrics is, that a decision maker can trade off error rates between groups. For example, instead of improving the the accuracy for the protected group, the decision maker could simply miss label some individuals of the unprotected group. Compared to disparate impact, error based metrics accept a perfect decision rule since it does not make errors.

More complex rules exist, but they are not covered here.

## 1.3. Long Term Fairness Setup
The above definitions are now extended for the sequential process. 
In this setup the bold capital letters $\bf{X}, \bf{A}, \bf{Y}, \bf{\hat{Y}}$ are aggregated over time with the first dimension representing time steps $t$. The superscript $^{(t)}$ indicates the referred time step. The protected features are considered to be constant over time.

$$ Y^{(t)} \in \mathbb{R}^{n}$$
$$ \hat{Y}^{(t)} \in \mathbb{R}^{n}$$
$$ X^{(t)} \in \mathbb{R}^{n \times m}$$

$$ A^{(t)} = \bf{A} \in \mathbb{R}^{n}$$

$$ \bf{Y} \in \mathbb{R}^{t \times n}$$
$$ \bf{\hat{Y}} \in \mathbb{R}^{t \times n}$$
$$ \bf{X} \in \mathbb{R}^{t \times n \times m}$$


# 2. Limits of Static Metrics
The above two methods are referred to as observational. They can be expressed as probability distributions over the random variables $X^{(t)}, Y^{(t)}, \hat{Y}^{(t)}$ and $A^{(t)}$. More limitations are discussed in the chapter [Inherent limitations of observational criteria](https://fairmlbook.org/pdf/causal.pdf) of the fair ML book. 

The limitations addressed here  are related to long term fairness. A decision made at some time point $t$ in this framework has an impact on the future.  Similar to the reasoning in [Delayed Impact](https://arxiv.org/abs/1803.04383), a static decision rule can have different impacts on the future depending on the assumptions about the data generation process.

The data and specifically the true labels $Y$ in the long term decision process are assumed to be distributed proportional to the number of past positive predictions for sub groups $G$ influencing certain individuals.

$$P(y_i^{(t)}=1) \sim \sum _ {j \in G} \sum _ {k=1} ^{n} \hat{y} _j ^{(t-k)} $$

$G$ is a degree of freedom and motivated by reasoning that the social environment has an impact on each individual. For instance, if parents graduated from university, their children are almost twice as likely to graduate themselves [](). Friends or public figures can also motivate someone by serving as role models. If members of a protected group are for instance accepted for university or employed in high positions other members of this group might be motivated to do so as well.

Long term effects are not explicitly considered in static metrics. Nevertheless, they can have impact on long term fairness. It is, however, not directly possible to apply static metrics to the long term data. One could for instance compute disparate impact on the aggregated data $\bf{X, Y, \hat{Y}, A}$, but this generalizes to the average disparate impact over each generation and does not take the dynamics into account. 

Consider a perfect decision function (one that makes no errors). From a static point of view, such a function could be considered as fair (i.e. it would satisfy all error based metrics). Under aboves assumption, such a decision rule  would never change the underlining data distribution. The number of individuals from each group labeled negative and positive would stay constant over time. However, if more individuals from the negative labeled group would have been given a chance, the overall number of positive labeled individuals would have increased.

It can be argued, that the decision marker itself has an interest in increasing the pool of qualified individuals.
In this setup, the effect of a static decision rule depends on the assumption about the underlining data generation. Two different assumptions about the group of individuals $G$ influencing the data generation process are implemented.

1. Positive predictions only influence the individual itself, which would be $G=\{i\}$.

2. The whole group benefits from positive predictions $G=\{j | a_j=a_i\}$.

They are discussed in the notebooks of chapter 2.

## 2.2 Defining Long Term Fairness
Until now it was only stated that static fairness metrics do not qualify for measuring long term fairness.
This is because the long term view implies other fairness notations beyond static criteria. Since the ability of individuals can improve as consequence of previous decisions, the question is shifted from *is an individual qualified today* to *could the individual be qualified in future*. The most intuitive way to define long term fairness would be, to state that the decision function must maximize the number of positive labeled individuals in future.

The number of true positive individuals at time $t$ is denoted as: $$ \# (Y^{(t)}=1)$$

The goal of this project is to visualize the effects of static decision functions on long term fairness. However, a long term metric is never explicitly discussed. Only the number of true positive individuals is counted. This is done by simulating two different data pipelines. One assuming that all previous predictions were positives referred to as baseline and the true data as produced by applying the decision function sequentially.

Note that simply counting the number of true positives is neither sufficient as longterm fairness measure. It for instance does not account for different necessary treatments between different individuals until a true positive label is achieved....



## 2.4 Related Work
Long term fairness has gained much interest recently. The most closely related framework is 

https://github.com/google/ml-fairness-gym long term fairness as markov decision process using reinforcement learning.



https://arxiv.org/abs/1909.09141 Causal Modeling for Fairness in Dynamical Systems

https://arxiv.org/abs/1803.04383 Delayed Impact of Fair Machine Learning

https://arxiv.org/abs/1903.01209 On the Long-term Impact of Algorithmic Decision Policies

https://arxiv.org/abs/1712.00064 A Short-term Intervention for Long-term Fairness in the Labor Market